Cеминар 2. Реализация кодирования по Хэммингу
-----------------------------------------------

***********************************************

*Материалы:*

[Википедия](https://ru.wikipedia.org/wiki/LZ77)  
[Викиконспекты](http://neerc.ifmo.ru/wiki/index.php?title=%D0%90%D0%BB%D0%B3%D0%BE%D1%80%D0%B8%D1%82%D0%BC%D1%8B_LZ77_%D0%B8_LZ78)  
[Хабр](https://habrahabr.ru/post/132683/)

***********************************************

In [1]:
def get_bits(list_result):
    bit1 = list_result[::2]
    bit2 = list_result[1::4] + list_result[2::4]
    bit4 = list_result[3::8] + list_result[4::8] + list_result[5::8] + list_result[6::8]
    bit8 = list_result[7:15]
    bit16 = list_result[15:]
    
    bit_1 = str(sum([int(x) for x in bit1]) % 2)
    bit_2 = str(sum([int(x) for x in bit2]) % 2)
    bit_4 = str(sum([int(x) for x in bit4]) % 2)
    bit_8 = str(sum([int(x) for x in bit8]) % 2)
    bit_16 = str(sum([int(x) for x in bit16]) % 2)
    
    return (bit_1, bit_2, bit_4, bit_8, bit_16)



sentence = 'Hello, World!'

if len(sentence) % 2:
    sentence += ' '
    
encrypt = ''

ab = list(sentence)

list_1 = list(sentence)[::2]
list_2 = list(sentence)[1::2]

for a, b in zip(list_1, list_2):
    
    # В 2 сс    
    a_order = ord(a)
    b_order = ord(b)
    result = '{0:08b}{1:08b}'.format(a_order, b_order)
    
    # Делаем список
    
    list_result = list(result)
    
    # Добавляем контрольные биты
    
    list_result.insert(0, '0')
    list_result.insert(1, '0')
    list_result.insert(3, '0')
    list_result.insert(7, '0')
    list_result.insert(15, '0')
    
    # Получаем контрольные биты

    result = get_bits(list_result)

    list_result[0] = result[0]
    list_result[1] = result[1]
    list_result[3] = result[2]
    list_result[7] = result[3]
    list_result[15] = result[4]
    
    encrypt += ''.join(list_result[:])

print encrypt

110010011000011000101000011001100011001100100011011111001001100110001010000010010111110011001111011010010010011001100011100100100101000001001000000


In [2]:
def get_2_char(list_result):
    list_result.pop(15)
    list_result.pop(7)
    list_result.pop(3)
    list_result.pop(1)
    list_result.pop(0)

    reverse_1 = ''.join(list_result[:8])
    reverse_2 = ''.join(list_result[8:])
    return (chr(int(reverse_1, 2)), chr(int(reverse_2, 2)))


# Вносим ошибки

# Разбиваем на участки по 21 биту

decoded = ''

for frag in zip(*[iter(encrypt)]*21):
    
    # Получаем контрольные биты
    result = get_bits(frag)
    
    # Находим позицию ошибки
    error = 0
    for index, i in enumerate([1, 2, 4, 8, 16]):
        if result[index] == '1':
            error += int(i)
    
    # Исправляем ошибку    
    if error:
        frag[error - 1] = str((int(frag[error - 1]) + 1) % 2)
        
    char_1, char_2 = get_2_char(list(frag))
        
    # Получаем char символы
    decoded += '{}{}'.format(char_1, char_2)
    
if decoded[-1] == ' ':
    decoded = decoded[:-1]
    
print decoded

Hello, World!
